In [27]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso 

# definirea modelelor 
linear_regression_model = LinearRegression() 
ridge_regression_model = Ridge(alpha=1) 
lasso_regression_model = Lasso(alpha=1) 

# calcularea valorii MSE și MAE 
from sklearn.metrics import mean_squared_error, mean_absolute_error

# mse_value = mean_squared_error(y_true, y_pred) 
# mae_value = mean_absolute_error(y_true, y_pred)

In [40]:
import numpy as np 
from sklearn.utils import shuffle 

# print the first 4 samples

print('The first 4 samples are:\n ', training_data[:4]) 
print('The first 4 prices are:\n ', prices[:4]) 

# shuffle 
training_data, prices = shuffle(training_data, prices, random_state=0)

The first 4 samples are:
  [[2.0170e+03 1.8351e+04 2.1900e+01 6.2400e+02 3.7480e+01 4.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 1.0000e+00]
 [2.0150e+03 9.1000e+04 2.1100e+01 8.1400e+02 5.5200e+01 5.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00]
 [2.0180e+03 2.0667e+04 1.8900e+01 1.1970e+03 8.1860e+01 5.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 1.0000e+00]
 [2.0070e+03 5.5000e+04 1.3200e+01 1.4950e+03 9.4000e+01 5.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00]]
The first 4 prices are:
  [3.41 2.5  7.55 2.65]


In [41]:
# 1
from sklearn import preprocessing

def normalizer(train_data, test_data = None):
    scaler = preprocessing.StandardScaler()
    scaler.fit(train_data)
    scaled_x_train = scaler.transform(train_data)
    
    if test_data is None:
        return scaled_x_train
    
    scaled_x_test = scaler.transform(test_data)
    return scaled_x_train, scaled_x_test

In [42]:
# 2
train_data = np.load('data/training_data.npy')
prices = np.load('data/prices.npy') 

train_data, prices = shuffle(train_data, prices, random_state=0)

num_samples = len(train_data) // 3

train_data1, prices1 = train_data[:num_samples], prices[:num_samples]
train_data2, prices2 = train_data[num_samples:2*num_samples], prices[num_samples:2*num_samples]
train_data3, prices3 = train_data[2*num_samples:], prices[2*num_samples:]


In [43]:
def train_test_model(model, train_data, train_label, test_data, test_label):
    train_data, test_data = normalizer(train_data, test_data)
    model.fit(train_data, train_label)
    
    y_pred = model.predict(test_data)
    y_true = test_label
    
    mse_value = mean_squared_error(y_pred, y_true) 
    mae_value = mean_absolute_error(y_pred, y_true)
    
    return mse_value, mae_value

 
linear_regression_model = LinearRegression()
mse1, mae1 = train_test_model(linear_regression_model, np.concatenate((train_data1, train_data2)),
                             np.concatenate((prices1, prices2)), train_data3, prices3)

mse2, mae2 = train_test_model(linear_regression_model, np.concatenate((train_data1, train_data3)),
                             np.concatenate((prices1, prices3)), train_data2, prices2)

mse3, mae3 = train_test_model(linear_regression_model, np.concatenate((train_data2, train_data3)),
                             np.concatenate((prices2, prices3)), train_data1, prices1)

MSE = (mse1 + mse2 + mse3) / 3
MAE = (mae1 + mae2 + mae3) / 3
print(MSE, MAE)

3.1656786600748696 1.3195819854736328


In [44]:
alpha = [1, 10, 100, 1000]

for a in alpha:
    ridge_regression_model = Ridge(alpha=a)
    mse1, mae1 = train_test_model(ridge_regression_model, np.concatenate((train_data1, train_data2)),
                             np.concatenate((prices1, prices2)), train_data3, prices3)

    mse2, mae2 = train_test_model(ridge_regression_model, np.concatenate((train_data1, train_data3)),
                                 np.concatenate((prices1, prices3)), train_data2, prices2)
    
    mse3, mae3 = train_test_model(ridge_regression_model, np.concatenate((train_data2, train_data3)),
                                 np.concatenate((prices2, prices3)), train_data1, prices1)

    MSE = (mse1 + mse2 + mse3) / 3
    MAE = (mae1 + mae2 + mae3) / 3
    print('alpha ', a, MSE, MAE)


alpha  1 3.167464256286621 1.3195734024047852
alpha  10 3.1673253377278647 1.3193697929382324
alpha  100 3.172309239705404 1.3185890515645344
alpha  1000 3.4331722259521484 1.3665776252746582


In [49]:
best_alpha = 10

model = Ridge(alpha=best_alpha)
train_data = normalizer(train_data)
model.fit(train_data, prices)
coef = model.coef_

print(coef)
np.argmax(np.abs(coef))

[ 1.6635205  -0.15533487 -0.4603492   0.40462962  1.3356842   0.13251224
 -0.08683416  0.          0.36666897 -0.36666256  0.          0.
 -0.22937056  0.22937447]


0